## Exercise - Pricing a Callable Bond

In [1]:
import warnings

import numpy as np
import pandas as pd
from scipy.stats import norm

warnings.filterwarnings('ignore')

## 1. Black’s Formula for Bond Options

### 1.1 Price the Vanilla (Non-Callable) Bond

**Theory:**  
The price of a vanilla coupon bond is the present value of all future cash flows, including coupons and the return of principal at maturity.

For a bond with maturity $T=3$ (years) and annual coupons, the cash flows occur at $t=1$, $t=2$, and $t=3$.

The pricing formula is:
$$
P_{vanilla} = \sum_{i=1}^{3} \text{CF}(t_i) \cdot DF(0, t_i)
$$

Where:  
- $\text{CF}(t_i)$ is the cash flow at time $t_i$  
- $DF(0, t_i)$ is the discount factor for time $t_i$ (from the provided data file)

**Cash flows for this bond ($N=100$, $cpn=6\%$):**
- $t=1$: \$6.00
- $t=2$: \$6.00
- $t=3$: \$106.00 (Coupon + Principal)

In [2]:
# Path to file containing the discount curve
data_path = "../data/discount_curve_2025-02-13.xlsx"

def calculate_vanilla_price(path):
    # Read the discount curve from the Excel file into a DataFrame
    df_curve = pd.read_excel(path)

    # Helper to fetch the discount factor for a specific time to maturity
    def get_df(t):
        return df_curve.loc[df_curve["ttm"] == t, "discount"].values[0]

    # Bond details
    face_value = 100
    coupon_rate = 0.06
    coupon = face_value * coupon_rate  # Annual coupon payment

    # Scheduled cash flows and their timings
    payment_times = np.array([1, 2, 3])
    cash_flows = np.array([coupon, coupon, coupon + face_value])

    # Collect discount factors corresponding to the payment times
    discount_factors = np.array([get_df(t) for t in payment_times])

    # Bond price is the sum of discounted cash flows
    bond_price = np.dot(cash_flows, discount_factors)
    return bond_price

# Calculate vanilla bond price
vanilla_price = calculate_vanilla_price(data_path)

# Create results DataFrame for display
vanilla_df = pd.DataFrame(
    {"Price": [f"{vanilla_price:.4f}"]},
    index=["Vanilla Bond Price"],
)
vanilla_df_caption = "1.1"
display(vanilla_df.style.set_caption(vanilla_df_caption))

,Price
Vanilla Bond Price,104.9867


### 1.2 Value of the Issuer's Call Option

**Theory:**  
Bond options are typically priced using Black's model (1976), which assumes the forward bond price follows a lognormal distribution. This is preferred over Black-Scholes for fixed income because it naturally incorporates the pull-to-par effect and the term structure of interest rates.

The value of a European call option on a bond is:

$$
C = DF(0, T_{\text{opt}}) \cdot [F \cdot N(d_1) - K \cdot N(d_2)]
$$

Where:

- $F = 103.31$ (Forward Clean Price)  
- $K = 100$ (Clean Strike)  
- $\sigma = 0.0268$ (Volatility)  
- $T_{\text{opt}} = 1.5$ (Option Expiration)  

With

$$
d_1 = \frac{\ln(F/K) + \frac{1}{2} \sigma^2 T_{\text{opt}}}{\sigma \sqrt{T_{\text{opt}}}}
$$

$$
d_2 = d_1 - \sigma \sqrt{T_{\text{opt}}}
$$